In [1]:
import urllib.request
import pandas as pd
import folium
from folium import plugins
import requests
import geopandas as gpd
from zipfile import ZipFile

## [Data Source](https://firms.modaps.eosdis.nasa.gov/active_fire/#firms-txt)

In [2]:

url = "https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/csv/MODIS_C6_1_USA_contiguous_and_Hawaii_7d.csv"  # URL of the file to download
filename = "data/downloaded_file.csv"  # Name of the file to save

try:
    urllib.request.urlretrieve(url, filename)
    print("File downloaded successfully.")
except Exception as e:
    print(f"An error occurred while downloading the file: {e}")

File downloaded successfully.


In [3]:
try:
    df = pd.read_csv(filename)
    print("CSV file loaded successfully.")
    
    # Perform operations on the dataframe here
    
except Exception as e:
    print(f"An error occurred while loading the CSV file: {e}")


CSV file loaded successfully.


In [4]:
url = "https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/shapes/zips/MODIS_C6_1_USA_contiguous_and_Hawaii_48h.zip"
zip_filename = "data/downloaded_shapefile.zip"
shapefile_filename = "data/MODIS_C6_1_USA_contiguous_and_Hawaii_48h.shp"

try:
    # Download the zip file
    response = requests.get(url)
    with open(zip_filename, 'wb') as zip_file:
        zip_file.write(response.content)
    
    print("Shapefile downloaded successfully.")
    
    # Extract the shapefile from the zip file
    with ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('data/')
    
    print("Shapefile extracted successfully.")
    
    # Read the shapefile into GeoPandas
    gdf = gpd.read_file(shapefile_filename)
    
    print("Shapefile loaded into GeoPandas successfully.")
    
    # Perform operations on the GeoDataFrame here
    
except Exception as e:
    print(f"An error occurred while downloading, extracting, or loading the shapefile: {e}")


Shapefile downloaded successfully.
Shapefile extracted successfully.
Shapefile loaded into GeoPandas successfully.


In [ ]:
# Create a Folium map centered on Chicago or your desired location
fire_map = folium.Map(location=[41.8781, -87.6298], zoom_start=3, tiles='cartodbdark_matter', no_wrap=True, attr='')

heat_data = [[[row['LATITUDE'],row['LONGITUDE']] for index,row in gdf.iterrows()]]

# Iterate over the points in the GeoDataFrame
for index, row in gdf.iterrows():
    # Get the coordinates of the point
    lat = row['LATITUDE']
    lon = row['LONGITUDE']
    
    # Create a custom marker icon
    custom_icon = folium.features.CustomIcon(icon_image='fire.png', icon_size=(30, 30))
    
    # Create a marker with the custom icon at the point's coordinates
    marker = folium.Marker(location=[lat, lon], icon=custom_icon)
    
    # Add the marker to the map
    marker.add_to(fire_map)


# Add a CSS style to change the background color to black
widget_style = """
<style>
    body {
        background-color: black !important;
    }
    @import url('https://fonts.googleapis.com/css2?family=Cinzel=swap');

    .map-title {
        color: white;
        font-family: 'Cinzel', serif;
        font-size: 20px;
        text-align: center;
    }
</style>
"""

# Add the CSS style to the HTML output
fire_map.get_root().html.add_child(folium.Element(widget_style))

# Add a title to the map with the map-title class
title_html = f'''
             <h3 align="center" class="map-title"><b>2023 US Fires (MODIS - Last 48 hours) <br> (Last Updated: {pd.to_datetime(gdf.ACQ_DATE).max().date()})</b></h3>
             '''
fire_map.get_root().html.add_child(folium.Element(title_html))

# Add a text annotation without a marker
folium.map.Marker(
    [30,-70],
    icon=folium.DivIcon(
        icon_size=(150, 30),
        icon_anchor=(0, 0),
        html='<div class="annotation">Alshival\'s Data Service<div class="subtext">support@alshival.com</div></div>'
    )
).add_to(fire_map)
css = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=Roboto&display=swap');

    .annotation {
        font-family: 'Roboto', sans-serif;
        font-size: 14pt;
        color: white;
        background-color: #e74c3c;
        padding: 8px 16px;
        border-radius: 4px;
        white-space: nowrap;
        display: inline-block;
        width: 250px;
        text-align: center;
    }
    
    .subtext {
        font-family: 'Roboto', sans-serif;
        font-size: 10pt;
        color: white;
        margin-top: 4px;
        text-align: center;
    }
</style>
"""
# Add the CSS style to the map
fire_map.get_root().html.add_child(folium.Element(css))
# Create the footer HTML content
footer_html = '''
<div style="text-align: center; font-size: 12px; color: #999999;">
    Data is collected from Nasa's MODIS satellite. It shows 48-hours worth of data reported by the system. Data the system reports with least 65% confidence are mapped.
</div>
'''

# Add the footer HTML to the map's HTML content
fire_map.get_root().html.add_child(folium.Element(footer_html))
fire_map.save("us_fires.html")
fire_map